In [2]:
%run Dataset_cleanup/cleanup_clinical.py

Clinical data set imported


 The following features do not provide any information: 
 ['Composite.Element.REF' 'ethnicity' 'gender' 'pathologicstage'
 'pathologyMstage' 'tumortissuesite'] 



 Variables that are not known at initial diagnosis: 
 ['daystodeath' 'daystolastfollowup' 'daystopsa' 'gleasonscore'
 'histologicaltype' 'numberoflymphnodes' 'pathologyTstage'
 'radiationtherapy' 'residualtumor' 'vitalstatus'] 


Variables that are known at the time of diagnosis:
 ['dateofinitialpathologicdiagnosis' 'psavalue' 'race' 'yearstobirth']

Dimensions of clinical dataframe: (499, 7)


In [3]:
%run Dataset_cleanup/cleanup_gene_counts.py

Gene Counts data set imported

Dimension of DataFrame: (550, 20501)

Transformation Successful!

Transcript abundance estimates have been transformed to transcripts per million reads


## Analysis of initial clinical presentation for  predictors of metastasis

In [4]:
"""The Clinical data (including metastasis label) have a multitude of missing items."""
print("Total observations in original dataset:",clinical.shape[0])

not_labeled = y[y.isnull()]
y_labels = y[y.notnull()]

print("\nLabeled observations:",y_labels.shape[0],"\nUnlabeled observations removed:",not_labeled.shape[0])


Total observations in original dataset: 499

Labeled observations: 426 
Unlabeled observations removed: 73


In [5]:
clinical = clinical.loc[y_labels.index]  #filter only observations where metastasis state is known
print("Filtering out missing metastasis state labels left",clinical.shape[0],"observations")

clinical.dropna(inplace = True) # Remove all observations where clinical data is missing
print("Removing NaN rows leaves ",clinical.shape[0]," observations in the feature set.") 

y_labels = y_labels.loc[clinical.index] #remove metastasis labels where no clinical data now exists due to NaN removal.

Filtering out missing metastasis state labels left 426 observations
Removing NaN rows leaves  359  observations in the feature set.


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer, accuracy_score

In [16]:
recaller = make_scorer(recall_score, greater_is_better=True, needs_proba=False, needs_threshold=False, pos_label='n1')
accuracy = make_scorer(accuracy_score, greater_is_better=True, needs_proba=False, needs_threshold=False)
clf = DecisionTreeClassifier(criterion='gini',
                            splitter='best',
                            max_depth=None,
                            min_samples_split=30,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features=None,
                            random_state=None,
                            max_leaf_nodes=None,
                            class_weight='balanced',
                            presort=False)
recall_vals = cross_val_score(clf, clinical, y_labels, scoring=recaller, cv=10)
accuracy_vals = cross_val_score(clf, clinical, y_labels, scoring=accuracy, cv=10)
print('The recall value for metastasis detection using a simple Decision Tree on patient information that is known at the onset of diagnosis is:')
print(recall_vals.mean())
print('The accuracy score for metasasis vs no metastasis is:\n',accuracy_vals.mean())
print("\nThe null error rate for metastasis detection on this dataset is:")
print((sum(y_labels == 'n1') / len(y_labels)))

The recall value for metastasis detection using a simple Decision Tree on patient information that is known at the onset of diagnosis is:
0.409523809524
The accuracy score for metasasis vs no metastasis is:
 0.511610896611

The null error rate for metastasis detection on this dataset is:
0.16991643454


In [17]:
clf.fit(clinical, y_labels)
print(pd.DataFrame({'Features':clinical.columns, 'Gini_Importance':clf.feature_importances_}))

                           Features  Gini_Importance
0  dateofinitialpathologicdiagnosis         0.271084
1                          psavalue         0.306299
2                      yearstobirth         0.339317
3                 race_Not_provided         0.000000
4                        race_asian         0.000000
5                  race_black_or_AA         0.000000
6                        race_white         0.083300
